## **ПРАКТИЧЕСКОЕ ЗАДАНИЕ**

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
 - Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
 - Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше.

In [ ]:
# импортируем библиотеки

import numpy as np
import pandas as pd
!pip install xlrd==1.2.0
!pip install pymorphy2
!pip install stop_words
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist
import warnings
warnings.filterwarnings('ignore')
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.layers import Embedding, MaxPooling1D, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

In [205]:
# вспомогательные функции

# функция обработки текста
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub('\sне', 'не', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)


# функция преобразования текста в последовательность
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

#### **ПРЕДОБРАБОТКА ДАННЫХ**

In [206]:
# загрузим отзывы

data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/NLP/отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [207]:
# обработаем отзывы

exclude = set(punctuation)
sw = set(get_stop_words('ru'))
morpher = MorphAnalyzer()

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [208]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [209]:
# разделим тексты на трейн и тест выборки

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [210]:
X_train.head()

547                                              классно
1863       невозможно использовать на рутованный телефон
13949                      работать хорошо без нарекание
11501                                            отлично
6238     ян мочь понять почему заблокировать мой аккаунт
Name: text, dtype: object

#### **СОЗДАНИЕ ТОКЕНОВ И ИХ ВЕКТОРИЗАЦИЯ**

In [211]:
# соберем отзывы в один текст и создадим токены

train_corpus = ' '.join(X_train)
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)

In [212]:
# отфильтруем цифро-буквенные токены

tokens_filtered = [word for word in tokens if word.isalnum()]

In [213]:
# зададим необходимые параметры, чтобы подготовить токены для работы с нейронной сетью

MAX_WORDS = 300
MAX_LEN = 100
NUM_CLASSES = 2

In [214]:
# создадим словарь с частотой употребления токенов
dist = FreqDist(tokens_filtered)

# возьмем топ самых частых токенов
tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]
tokens_filtered_top[:10]

['приложение',
 'всё',
 'и',
 'очень',
 'удобно',
 'в',
 'я',
 'на',
 'работать',
 'удобный']

In [215]:
# создадим словарь для обучения

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [216]:
# конвертируем текст в последовательность

X_train_arr = np.asarray([text_to_sequence(text, MAX_LEN) for text in X_train], dtype=np.int32)
X_test_arr = np.asarray([text_to_sequence(text, MAX_LEN) for text in X_test], dtype=np.int32)

In [217]:
X_train_arr.shape

(15798, 100)

In [218]:
X_train_arr[100]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 298,  23,
       121,  67,  93,  28, 299,  31,  18,   2,  70], dtype=int32)

In [219]:
# преобразуем таргет

y_train_c = keras.utils.np_utils.to_categorical(y_train, NUM_CLASSES)
y_test_c = keras.utils.np_utils.to_categorical(y_test, NUM_CLASSES)

#### **ОБУЧЕНИЕ НЕЙРОСЕТИ С НАТРЕНИРОВАННЫМИ ВЕСАМИ**

Модель: ruwikiruscorpora_upos_cbow_300_10_2021

https://rusvectores.org/ru/models/

In [ ]:
# скачаем модель с преобученными весами
# ruwikiruscorpora_upos_cbow_300_10_2021

!wget http://vectors.nlpl.eu/repository/20/220.zip
!unzip /content/220.zip

In [221]:
# Создадим словарь из токенов и их предобученных весов

embeddings_index = {}
with open('/content/model.txt') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 249334 word vectors.


In [222]:
# зададим необходимые параметры для работы с нейронной сетью

EMBEDDING_DIM = 256
EPOCHS = 10
BATCH_SIZE = 256
PRINT_BATCH_N = 100

In [223]:
# вычислим матрицу векторов

embedding_matrix = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [224]:
# построим архитектуру сети

model = Sequential()
model.add(Embedding(len(vocabulary) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_LEN, trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(50))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

In [225]:
# скомпилируем модель

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [226]:
# проведем обучение модели

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train_arr, y_train_c,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(X_test_arr, y_test_c),
                    callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/10
62/62 [==============================] - 18s 286ms/step - loss: 0.6728 - accuracy: 0.8351 - val_loss: 0.6526 - val_accuracy: 0.8468
Epoch 2/10
62/62 [==============================] - 18s 298ms/step - loss: 0.6349 - accuracy: 0.8469 - val_loss: 0.6172 - val_accuracy: 0.8468


**ВЫВОД:**

Максимальная точность на тесте - $0.8468$.

#### **ОБУЧЕНИЕ НЕЙРОСЕТИ БЕЗ НАТРЕНИРОВАННЫХ ВЕСОВ**

In [227]:
# построим архитектуру сети

model = Sequential()
model.add(Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(50))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

In [228]:
# скомпилируем модель

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [229]:
# проведем обучение модели

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train_arr, y_train_c,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(X_test_arr, y_test_c),
                    callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/10
62/62 [==============================] - 14s 215ms/step - loss: 0.3562 - accuracy: 0.8541 - val_loss: 0.1966 - val_accuracy: 0.9071
Epoch 2/10
62/62 [==============================] - 14s 222ms/step - loss: 0.1854 - accuracy: 0.9196 - val_loss: 0.1802 - val_accuracy: 0.9218


In [230]:
# score = model.evaluate(X_test_arr, y_test_c, batch_size=BATCH_SIZE, verbose=1)
# print('\n')
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# results = model.predict(X_test_arr, batch_size=BATCH_SIZE, verbose=1)

**ВЫВОД:**

Максимальная точность на тесте - $0.9218$.

Со слоем Embedding() по умолчанию удалось добиться точности выше. Возможно, это связано с тем, что предобученные веса не подходят по тематике к отзывам.

Using pre-trained word embeddings in a Keras model

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html